<a href="https://www.kaggle.com/code/hriteshmaikap/neural-network-1-1?scriptVersionId=221781459" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import torch
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/thyroid-cancer-risk-prediction-dataset/thyroid_cancer_risk_data.csv
/kaggle/input/alzheimers-prediction-dataset-global/alzheimers_prediction_dataset.csv


## Simple Neural Network from Scratch

In [2]:
import numpy as np # linear algebra
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("/kaggle/input/alzheimers-prediction-dataset-global/alzheimers_prediction_dataset.csv")
df.head()

,Country,Age,Gender,Education Level,BMI,Physical Activity Level,Smoking Status,Alcohol Consumption,Diabetes,Hypertension,...,Dietary Habits,Air Pollution Exposure,Employment Status,Marital Status,Genetic Risk Factor (APOE-ε4 allele),Social Engagement Level,Income Level,Stress Levels,Urban vs Rural Living,Alzheimer’s Diagnosis
0,Spain,90,Male,1,33.0,Medium,Never,Occasionally,No,No,...,Healthy,High,Retired,Single,No,Low,Medium,High,Urban,No
1,Argentina,72,Male,7,29.9,Medium,Former,Never,No,No,...,Healthy,Medium,Unemployed,Widowed,No,High,Low,High,Urban,No
2,South Africa,86,Female,19,22.9,High,Current,Occasionally,No,Yes,...,Average,Medium,Employed,Single,No,Low,Medium,High,Rural,No
3,China,53,Male,17,31.2,Low,Never,Regularly,Yes,No,...,Healthy,Medium,Retired,Single,No,High,Medium,Low,Rural,No
4,Sweden,58,Female,3,30.0,High,Former,Never,Yes,No,...,Unhealthy,High,Employed,Married,No,Low,Medium,High,Rural,No


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74283 entries, 0 to 74282
Data columns (total 25 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Country                               74283 non-null  object 
 1   Age                                   74283 non-null  int64  
 2   Gender                                74283 non-null  object 
 3   Education Level                       74283 non-null  int64  
 4   BMI                                   74283 non-null  float64
 5   Physical Activity Level               74283 non-null  object 
 6   Smoking Status                        74283 non-null  object 
 7   Alcohol Consumption                   74283 non-null  object 
 8   Diabetes                              74283 non-null  object 
 9   Hypertension                          74283 non-null  object 
 10  Cholesterol Level                     74283 non-null  object 
 11  Family History 

## Pre-processing the data

In [5]:
categorical_columns = [
    'Country', 'Gender', 'Physical Activity Level', 'Smoking Status', 
    'Alcohol Consumption', 'Diabetes', 'Hypertension', 'Cholesterol Level', 
    'Family History of Alzheimer’s', 'Depression Level', 'Sleep Quality', 
    'Dietary Habits', 'Air Pollution Exposure', 'Employment Status', 
    'Marital Status', 'Genetic Risk Factor (APOE-ε4 allele)', 
    'Social Engagement Level', 'Income Level', 'Stress Levels', 
    'Urban vs Rural Living', 'Alzheimer’s Diagnosis'
]

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save the encoder for potential inverse transform later

numerical_columns = ['Age', 'Education Level', 'BMI', 'Cognitive Test Score']

scaler = MinMaxScaler()

df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

print(df.head())

   Country       Age  Gender  Education Level       BMI  \
0       16  0.909091       1         0.052632  0.878788   
1        0  0.500000       1         0.368421  0.690909   
2       14  0.818182       0         1.000000  0.266667   
3        4  0.068182       1         0.894737  0.769697   
4       17  0.181818       0         0.157895  0.696970   

   Physical Activity Level  Smoking Status  Alcohol Consumption  Diabetes  \
0                        2               2                    1         0   
1                        2               1                    0         0   
2                        0               0                    1         0   
3                        1               2                    2         1   
4                        0               1                    0         1   

   Hypertension  ...  Dietary Habits  Air Pollution Exposure  \
0             0  ...               1                       0   
1             0  ...               1                  

In [6]:
df.shape

(74283, 25)

In [7]:
X = df.drop('Alzheimer’s Diagnosis',axis=1).values
y = df['Alzheimer’s Diagnosis'].values

In [8]:
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=41)

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(59426, 24)
(59426,)
(14857, 24)
(14857,)


In [10]:
# converting train and test set to float tensor
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

## Buildin the Neural Network

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Define the neural network architecture.
class AlzheimerNet(nn.Module):
    def __init__(self, input_size, hidden1=12, hidden2=6):
        super(AlzheimerNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden1)
        self.layer2 = nn.Linear(hidden1, hidden2)
        self.output = nn.Linear(hidden2, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.output(x)
        return self.sigmoid(x)

# Initialize TensorBoard SummaryWriter.
writer = SummaryWriter("runs/hparam_tuning")

# Define the hyperparameter grid.
learning_rates = [0.001, 0.005, 0.01]
epochs_list = [50, 100, 150]

# To store the results of each hyperparameter combination.
results = []
best_accuracy = 0.0
best_params = {}

# Global run counter (for unique naming in TensorBoard).
global_run = 0

# Loop over each hyperparameter combination.
for lr in learning_rates:
    for num_epochs in epochs_list:
        # Define a unique name for this run to help distinguish logs in TensorBoard.
        run_name = f"run_{global_run}_lr_{lr}_epochs_{num_epochs}"
        print(f"\nTraining with learning rate = {lr} and epochs = {num_epochs}")

        # Reinitialize the model for the current hyperparameter combination.
        input_size = X_train.shape[1]  # Number of features in the training set.
        model = AlzheimerNet(input_size)
        
        # Define loss and optimizer with the current learning rate.
        criterion = nn.BCELoss()  # Binary Cross Entropy Loss.
        optimizer = optim.Adam(model.parameters(), lr=lr)
        
        # Training loop.
        for epoch in range(num_epochs):
            model.train()            # Set the model to training mode.
            optimizer.zero_grad()    # Clear previous gradients.
            
            # Forward pass: compute predictions and loss.
            outputs = model(X_train)
            loss = criterion(outputs.squeeze(), y_train)
            
            # Backward pass and optimization.
            loss.backward()
            optimizer.step()
            
            # Log the training loss for the current epoch.
            writer.add_scalar(f"Loss/train_{run_name}", loss.item(), epoch)
        
        # Evaluation phase.
        model.eval()  # Set model to evaluation mode.
        with torch.no_grad():
            y_pred = model(X_test)
            # Convert probabilities to binary predictions using 0.5 threshold.
            y_pred = (y_pred >= 0.5).float()
            
            # Compute test accuracy.
            accuracy = accuracy_score(y_test, y_pred)
            print(f"Test Accuracy: {accuracy:.4f}")
            
            # Log the test accuracy.
            writer.add_scalar(f"Accuracy/test_{run_name}", accuracy, 0)
            
            # Log hyperparameters and corresponding metric using add_hparams.
            hparams = {'lr': lr, 'epochs': num_epochs}
            metrics = {'accuracy': accuracy}
            writer.add_hparams(hparam_dict=hparams, metric_dict=metrics, run_name=run_name)
            
            # Store results.
            results.append({'learning_rate': lr, 'epochs': num_epochs, 'accuracy': accuracy})
            
            # Update best hyperparameters if needed.
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {'learning_rate': lr, 'epochs': num_epochs}
            
            # Increment the global run counter.
            global_run += 1

# Print a summary of all hyperparameter tuning results.
print("\nHyperparameter Tuning Results:")
for res in results:
    print(f"Learning Rate: {res['learning_rate']}, Epochs: {res['epochs']}, Accuracy: {res['accuracy']:.4f}")

print(f"\nBest Hyperparameters: Learning Rate = {best_params['learning_rate']}, "
      f"Epochs = {best_params['epochs']}, with Accuracy = {best_accuracy:.4f}")

# Close the TensorBoard writer.
writer.close()


Training with learning rate = 0.001 and epochs = 50
Test Accuracy: 0.5861

Training with learning rate = 0.001 and epochs = 100
Test Accuracy: 0.5888

Training with learning rate = 0.001 and epochs = 150
Test Accuracy: 0.6152

Training with learning rate = 0.005 and epochs = 50
Test Accuracy: 0.6781

Training with learning rate = 0.005 and epochs = 100
Test Accuracy: 0.7119

Training with learning rate = 0.005 and epochs = 150
Test Accuracy: 0.7096

Training with learning rate = 0.01 and epochs = 50
Test Accuracy: 0.6985

Training with learning rate = 0.01 and epochs = 100
Test Accuracy: 0.7106

Training with learning rate = 0.01 and epochs = 150
Test Accuracy: 0.7118

Hyperparameter Tuning Results:
Learning Rate: 0.001, Epochs: 50, Accuracy: 0.5861
Learning Rate: 0.001, Epochs: 100, Accuracy: 0.5888
Learning Rate: 0.001, Epochs: 150, Accuracy: 0.6152
Learning Rate: 0.005, Epochs: 50, Accuracy: 0.6781
Learning Rate: 0.005, Epochs: 100, Accuracy: 0.7119
Learning Rate: 0.005, Epochs: 15